# Creating a custom dataset class to get all csv files from folder

In [18]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import glob
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [81]:
class StocksDataSet(Dataset):
    def __init__(self, root_directory):
        self.root = root_directory
        self.stocks_files = glob.glob(f"{self.root}/**/*.csv", recursive=True)
        self.test_scaler = StandardScaler()

        self.mm_scaler = MinMaxScaler()
        #self.transform = transforms.Compose([transforms.ToTensor()])
        # for i in self.stocks_files:
        #     print(i)
      

    def __len__(self):
        return len(self.stocks_files)

    def test_func(self, input_data, output_data, steps_for_input, steps_for_output):
        X,y = [], []
        
        for i in range(len(input_data)):
            end_x = i + steps_for_input
           # print(end_x)
            output_x = end_x + steps_for_output - 1
            if output_x > len(input_data): 
                break
            #print(output_data[end_x - 1 : output_x])
            seq_x, seq_y = input_data[i : end_x], output_data[end_x - 1 : output_x]
            X.append(seq_x), y.append(seq_y)
    
    # for i in range(X.shape[1]):
        
    #return scaler_test.fit_transform(X), scaler_test.transform(y)
        #print(seq_x)
        #print(seq_y)
        return np.array(X), np.array(y)

    def __getitem__(self, stock_index):
      
        data = pd.read_csv(self.stocks_files[stock_index], index_col="Date", parse_dates = True)
        
        data.replace([np.inf, -np.inf], np.nan, inplace = True)
        data.dropna(inplace = True)
        close = data["Close"]
        related_features = data.drop("Close", axis = 1)
        total_data = len(data)
        test_split = round(0.80 * total_data)
        
        X_calc, y_calc = self.test_func(related_features, close, 90, 14)
        
        X_train = torch.Tensor(X_calc[:test_split])
        X_test = torch.Tensor(X_calc[test_split:])
        if X_test.size(dim = 0) == 0:
            return np.zeros((0, 0)), np.zeros((0, 0)),np.zeros((0, 0)), np.zeros((0, 0))
        y_train = torch.Tensor(y_calc[:test_split])
        y_test = torch.Tensor(y_calc[test_split:])
            
        X_train = torch.Tensor(self.test_scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape))
        X_test = torch.Tensor(self.test_scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape))
        
            
        y_train = torch.Tensor(self.mm_scaler.fit_transform(y_train.reshape(-1, y_train.shape[-1])).reshape(y_train.shape))
        y_test = torch.Tensor(self.mm_scaler.transform(y_test.reshape(-1, y_test.shape[-1])).reshape(y_test.shape))
           
        return X_train, X_test, y_train, y_test
        

In [83]:
# Testing
test = StocksDataSet("../Custom LSTM Model/Data/Stocks Data")
#stocks_test = StocksDataSet(test)

gen = DataLoader(test)
for i, j,k,l in gen:
    pass